In [ ]:
!pip install tensorflow-text

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download CellPhones -f CellPhonesRating.csv

In [4]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow.keras import datasets, layers, models

In [227]:
df = pd.read_csv("/content/CellPhonesRating.csv.zip")
df = df.dropna(how='any',axis=0)
df = df.drop_duplicates(subset =['productID', 'reviewerID'] , keep = 'first')

In [228]:
df = df[:100000]

In [224]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe.groupby(itemID).count()['Unnamed: 0'].values:
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from lfoat to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return df_user_item

In [229]:
user_item_matrix = DatasetToUserItemDataFrame(df,'reviewerID','productID','rating')

In [ ]:
user_item_matrix

In [220]:
#Train/Test data split
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(user_item_matrix)

In [ ]:
encoder_input = layers.Input(shape=(),name='user_item')
